In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
import seaborn as sb

%matplotlib inline


### Leitura do Dataset

In [ ]:
dados = pd.read_csv('../input/SEMANAL_BRASIL-DESDE_2013.csv', decimal=",")

# Análise Exploratória no DATASET (EDA)
Veremos respectivamente...
~~~~
1- Quantidade de linhas e colunas
2- Descrição do Index
3- Quais colunas estão presentes no Dataframe
4 -Contagem de dados não-nulos
~~~~


In [ ]:
dados.shape #Linhas x Colunas

In [ ]:
dados.index # Descrição Index

In [ ]:
dados.columns #Colunas presentes

In [ ]:
dados.count() #Total dados não-nulos

### Declarando variável para armazenar somenta a coluna de ***NÚMERO DE POSTOS PESQUISADOS***, ***DATA INICIAL*** e ***DATA FINAL*****.

In [ ]:
numeroPostosPesquisados = dados.iloc[0:1824, 3]
semanaInicial = dados.iloc[0:1824, 0]
semanaFinal = dados.iloc[0:1824, 1]

Seperamos quais os tipos de produtos foram utilizados na pesquisa **colocamos no index** e com isso mostramos a tabela em função dos **PRODUTOS**

In [ ]:
tipoCombustivel = dados.set_index('PRODUTO')
tipoCombustivel.head()

Definimos agora somente os dados referente ao combustivel do tipo **GASOLINA COMUM**

In [ ]:
gasolina = tipoCombustivel.loc['GASOLINA COMUM']

In [ ]:
gasolina


### Obtendo ***média***, ***média truncada***, ***valor máximo***, ****valor mínimo*** do número de postos pesquisados para o tipo de combustivel "GASOLINA COMUM".

In [ ]:
numeroPP = gasolina.iloc[0:304 , 2]

In [ ]:
valMax = numeroPP.max() 
print(valMax,"----> Número maximo de postos onde foram pesquisados o valor em R$, da gasolina")

In [ ]:
valMin = numeroPP.min() 
print(valMin,"----> Número mínimo de postos onde foram pesquisados o valor em R$, da gasolina")

In [ ]:
media = numeroPP.mean() 

In [ ]:
print("{0:.0f}".format(round(media)),"----> Média de postos onde foram pesquisados o valor em R$, da gasolina ") 

# Média truncada, Desvio Padrão, Histograma

In [ ]:
histograma = numeroPP.hist()
print(histograma, "@@@ Esse é o gráfico de número de postos pesquisados em função do nº de postos pesquisados")

In [ ]:
desvioPadrao = numeroPP.std()
print("{0:.0f}".format(round(desvioPadrao)),"----> Valor de desvio padrão ") 

In [ ]:
mTruncada = valMax + valMin - media
print("{0:.0f}".format(round(mTruncada)),"----> Valor da Média Truncada ") 

# Validação

In [ ]:
gasolina.describe()

# Aplicando K-Means


In [ ]:
dadosKM = dados.iloc[0:1824, [8,14]]
dadosKM.replace(".",",")
dadosKM_array = dadosKM.values
kmeans = KMeans(n_clusters=5, init='k-means++', n_init=10, random_state=1234)
dadosKM["clusters"] = kmeans.fit_predict(dadosKM_array)
dadosKM.groupby("clusters").aggregate("mean").plot.bar(figsize=(10,7.5))
plt.title("(R$ Máximo) Revenda x Distribuição")

In [ ]:
dadosKM

Nota-se que em todos clusters(grupos) formados o preço máximo de revenda foi superior ao preço máximo de distribuição

In [ ]:
gasolinaKM = gasolina.iloc[0:1824, [7,13]]
gasolinaKM_array = gasolinaKM.values
kmeans = KMeans(n_clusters=5, init='k-means++', n_init=10, random_state=1234)
gasolinaKM["clusters"] = kmeans.fit_predict(gasolinaKM_array)
gasolinaKM.groupby("clusters").aggregate("mean").plot.bar(figsize=(10,7.5))
plt.title("R$ REVENDA X DISITRIBUIÇÃO (GASOLINA) ")

In [ ]:
gasolinaKM

#### Note a formação da ultima coluna "clusters" onde aleatoriamente foram distribuidos registros em 5 grupos (0,1,2,3,4) e com isso a plotagem gráfica nos mostra as chances em que determinado grupo tem de ter o preço máximo de revenda maior que de distribuição (nulo)

# Aplicando KNN

#### O Algoritmo foi aplicado encima dos resultados obtido com o pré-processamento dos dados e aplicação do K-means


#### Declarando 2 variáveis X e Y 
onde **X** recebe os atributos (PREÇO MÁXIMO REVENDA, PREÇO MÁXIMO DISTRIBUIÇÃO) E **Y** recebe os rotulos ( clusters formados ) 

In [ ]:
x = gasolinaKM.iloc[:, [0,1]].values
y = gasolinaKM.iloc[:, 2].values

### Divisão em conjunto de teste e de treino
Para evitar o ajuste excessivo, dividiremos nosso conjunto de dados em duas partes, uma para treinamento e outra para teste e avaliar o modelo.
Desta forma, nosso algoritmo é testado em dados não vistos, como seria em uma aplicação em **produção**.
O código abaixo divide o conjunto de dados em 80% de dados de treino e 20% de dados de teste.

In [ ]:
from sklearn.model_selection import train_test_split  
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20)

Logo abaixo veremos o pacote de pré-processamento da Scikit-Learn tem algumas funções de padronização como a **StandardScaler()**
O Ele ignora a forma da distribuição e transforma os dados para forma com média próxima de zero e um desvio padrão próximo a um.

In [ ]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(x_train)

x_train = scaler.transform(x_train)  
x_test = scaler.transform(x_test) 

O método **fit()** calcula a média e o desvio padrão da distribuição para fazer a padronização dos dados.
E método **transform()** aplica os cálculos para fazer a transformação nos dados.

### Treinamento e Previsões

In [ ]:
from sklearn.neighbors import KNeighborsClassifier  # import do KNN
classifier = KNeighborsClassifier(n_neighbors = 5)  # parametro = 5
classifier.fit(x_train, y_train)  

Previsao dos dados de teste

In [ ]:
y_pred = classifier.predict(x_test)

### Avaliando o algoritmo
Para avaliar um algoritmo, a matriz de confusão, a precisão, o recall e a pontuação f1 são as **métricas** mais utilizadas.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix  # import da matriz de confusão e report
print(confusion_matrix(y_test, y_pred)) # imprime a matriz de confusão
print(classification_report(y_test, y_pred)) # imprime as métricas

Os resultados mostram que o algoritmo KNN foi capaz de classificar 61 registros do conjunto de teste com quase 100% de precisão

In [ ]:
classifier = KNeighborsClassifier(n_neighbors = 18)
classifier.fit(x_train, y_train)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Apos ajustar o numero de Neighbors para 18 vimos que não obtivemos 100% de precisão... isso acontece porque o KNN nem sempre funciona tão bem com recursos de alta dimensionalidade ou categóricos.